# Inteligentny system rekomendacyjny
---

### **Opis**:
W celu poznania potrzeb swoich klientów firma zdecydowała się na zaprojektowania systemu rekomendacyjnego. Aby znaleźć rozwiązanie musimy przyjąć założenie, że podobni klienci są zainteresowani podobnymi produktami. Aby przedstawić system został użyty zbiór danych z: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop

---

## **Part 0: Importing frameworks**

In [20]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [3]:
cosmetics = pd.read_csv("C:/Users/e u n o i a/Downloads/ratings_Beauty.csv/ratings_Beauty.csv")
cosmetics.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [4]:
cosmetics.describe()

,Rating,Timestamp
count,2.023070e+06,2.023070e+06
mean,4.149036e+00,1.360389e+09
std,1.311505e+00,4.611860e+07
min,1.000000e+00,9.087552e+08
25%,4.000000e+00,1.350259e+09
50%,5.000000e+00,1.372810e+09
75%,5.000000e+00,1.391472e+09
max,5.000000e+00,1.406074e+09


In [5]:
cosmetics.columns

Index(['UserId', 'ProductId', 'Rating', 'Timestamp'], dtype='object')

In [7]:
agg_ratings = cosmetics.groupby('ProductId').agg(mean_rating = ('Rating', 'mean'))

In [17]:
cosmetics= cosmetics.head(10000)

In [40]:
ratings_utility_matrix = cosmetics.pivot_table(values='Rating', index='UserId', columns='ProductId', fill_value=0)
ratings_utility_matrix.fillna(0, inplace=True)
ratings_utility_matrix.head()


ProductId,0205616461,0558925278,0733001998,0737104473,0762451459,1304139212,1304139220,130414089X,130414643X,1304146537,...,B000052YPE,B000052YPF,B000052YPG,B000052YPH,B000052YPM,B000052YPU,B000052YPV,B000052YPY,B000052YQ0,B000052YQ2
UserId,,,,,,,,,,,,,,,,,,,,,
A00205921JHJK5X9LNP42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A024581134CV80ZBLIZTZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03056581JJIOL5FSKJY7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03099101ZRK4K607JVHH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0505229A7NSH3FRXRR4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
X = ratings_utility_matrix.T
X.head()

UserId,A00205921JHJK5X9LNP42,A024581134CV80ZBLIZTZ,A03056581JJIOL5FSKJY7,A03099101ZRK4K607JVHH,A0505229A7NSH3FRXRR4,A05492663T95KW63BR75K,A059547920Q3LZVFHLPI3,A07410232KYRFR25CIUGJ,A082796624UNM47DSAI6K,A0864963DOAY7LXGS5I6,...,AZW1HXXYAC15B,AZWRTJPN7NXT,AZWTXHXZXFAYP,AZYQEFB9Y5N22,AZZHB6U54UDYW,AZZHJZP4GQPPZ,AZZNK89PXD006,AZZOFVMQC0BJG,AZZQXL8VDCFTV,AZZTJQ7CQZUD8
ProductId,,,,,,,,,,,,,,,,,,,,,
0205616461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0558925278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0733001998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0737104473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0762451459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
correlation_matrix = np.corrcoef(decomposed_matrix)
decomposed_matrix.shape

(886, 10)

In [44]:
import random
i = random.randint(0,correlation_matrix.shape[0])
#na stronie i to produkt klienta
product_ID = X.index[i]

In [45]:
correlation_ProductId = correlation_matrix[i]
recommended_products = X.index[correlation_ProductId > 0.90] 
recommended_products = list(recommended_products)

#Top 10 Products recommended to the user based on the purchase done by user
recommended_products[0:10]

['1304196070',
 '1304351475',
 '1304622622',
 '1929099886',
 '1987958179',
 '3222000565',
 '322700075X',
 '3227001381',
 '3423470476',
 '4057362843']

In [153]:
original_data = pd.read_csv("C:/Users/e u n o i a/Downloads/archive(7)/nyka_top_brands_cosmetics_product_reviews.csv")

In [154]:
data = original_data

In [155]:
data.columns

Index(['product_id', 'brand_name', 'review_id', 'review_title', 'review_text',
       'author', 'review_date', 'review_rating', 'is_a_buyer', 'pro_user',
       'review_label', 'product_title', 'mrp', 'price', 'product_rating',
       'product_rating_count', 'product_tags', 'product_url'],
      dtype='object')

In [156]:
agg_ratings = data.groupby('product_id').agg(mean_rating = ('product_rating', 'mean'))
ratings_utility_matrix = data.pivot_table(values='product_rating', index='author', columns='product_id', fill_value=0)
ratings_utility_matrix.fillna(0, inplace=True)
ratings_utility_matrix.head()


product_id,250,479,981,1017,1139,1268,1290,5612,8078,9471,...,6583638,6599370,6599374,6599375,6604346,6604347,6836720,7066986,7308001,7749427
author,,,,,,,,,,,,,,,,,,,,,
Monika,0.0,0.0,0.0,0,0.0,4.1,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0
# Ho,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0
#Udhyama# Bunga,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0
$weety Soni,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0
*,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0


In [157]:
X = ratings_utility_matrix.T
X.head()

author,Monika,# Ho,#Udhyama# Bunga,$weety Soni,*,.,..,...,03 28,036_15_shalini s,...,å½¡Shreya Sinha,å½¡Shreya SinhaRoy,Šőmă Věřmä,मेघा भट्ट,रचना सिंह,सुप्रिया श्रीवास्तवा,তনুশ্রী পাল,শ্রাবণী সাহা,সুচরিতা সরকার,ਹਰਪੀ੍ਤ ਕੌਰ
product_id,,,,,,,,,,,,,,,,,,,,,
250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.1,0.0,0.0,4.1,0.0,0.0,0.0,0.0,0.0
1017,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
correlation_matrix = np.corrcoef(decomposed_matrix)
decomposed_matrix.shape
import random
i = random.randint(0,correlation_matrix.shape[0])
#na stronie i to produkt klienta
product_ID = X.index[i]

In [184]:
correlation_ProductId = correlation_matrix[i]
recommended_products = X.index[correlation_ProductId > 0.90] 
recommended_products = list(recommended_products)

#Top 10 Products recommended to the user based on the purchase done by user
recommended_products = recommended_products[0:10]

In [185]:
original_data.tail()

,product_id,brand_name,review_id,review_title,review_text,author,review_date,review_rating,is_a_buyer,pro_user,review_label,product_title,mrp,price,product_rating,product_rating_count,product_tags,product_url
61279,1044490,Herbal Essences,17769058,Good,I use this thrice a week. Has helped me in mak...,Radhesh,2021-04-12 13:28:41,5.0,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...
61280,1044490,Herbal Essences,17766838,Great,This shampoo has a amazing fragnance and textu...,Shalini Sirohi,2021-04-12 11:47:07,5.0,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...
61281,1044490,Herbal Essences,17765866,Really mild and nice,"Used it for the first time, feels really fresh...",Sana kanted,2021-04-12 10:58:34,5.0,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...
61282,1044490,Herbal Essences,17761057,Amazing product!,One of the best shampoo which are sulphate and...,neeru chouhan,2021-04-11 21:44:55,5.0,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...
61283,4633369,Colorbar,25987603,Not good,It is so irritating so bad i don't like this much,Mssuhv,2022-06-20 17:41:52,2.0,False,False,NaN,Colorbar Mogra Mist - Scent Of An India Temple...,595,417,1.5,2,NaN,https://www.nykaa.com/colorbar-mogra-mist-scen...


In [186]:
title = original_data["product_title"]
title[37267]

"L'Oreal Paris Matte Signature Eyeliner"

In [187]:
print("because you chose \"" + title[i] + "\" you may also like ")
for j in range(len(recommended_products)-1):
    print(title[j])

because you chose "Olay Eye Cream - With Niacinamide & Pentapeptides" you may also like 
Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF)
Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF)
Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF)
Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF)
Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF)
Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF)
Olay Regenerist Whip Mini and Ultimate Eye Cream Combo
Olay Regenerist Whip Mini and Ultimate Eye Cream Combo
Olay Regenerist Whip Mini and Ultimate Eye Cream Combo


In [188]:
recommended_products

[250, 1017, 5612, 9471, 36379, 60541, 64110, 68508, 139993, 173656]